# Using PASTE and DLPFC as an example here

In [ ]:
import numpy as np
import sklearn
import sklearn.metrics.pairwise
import pandas as pd


def cal_layer_based_alignment_result(alignment, labels):
    res = []
    l_dict = {"0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6}
    cnt0 = 0
    cnt1 = 0
    cnt2 = 0
    cnt3 = 0
    cnt4 = 0
    cnt5 = 0
    cnt6 = 0
    for i, elem in enumerate(alignment):
        if labels[i] == '-1' or labels[elem.argmax() + alignment.shape[0]] == '-1':
            continue 
        if l_dict[labels[i]] == l_dict[labels[elem.argmax() + alignment.shape[0]]]:
            cnt0 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 1:
            cnt1 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 2:
            cnt2 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 3:
            cnt3 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 4:
            cnt4 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 5:
            cnt5 += 1
        if abs(l_dict[labels[i]] - l_dict[labels[elem.argmax() + alignment.shape[0]]]) == 6:
            cnt6 += 1
    print(alignment.shape[0])
    print(cnt0/alignment.shape[0], cnt1/alignment.shape[0], cnt2/alignment.shape[0], cnt3/alignment.shape[0], cnt4/alignment.shape[0], cnt5/alignment.shape[0], cnt6/alignment.shape[0])
    res.extend([cnt0/alignment.shape[0], cnt1/alignment.shape[0], cnt2/alignment.shape[0], cnt3/alignment.shape[0], cnt4/alignment.shape[0], cnt5/alignment.shape[0], cnt6/alignment.shape[0]])
    return res

In [ ]:
import os

def load_paste(path_='../samples/paste', sec='1515107_1515108'):
    alignment=np.load(os.path.join(path_, sec, "iter0embedding.npy"))
    labels=np.load(os.path.join(path_, sec, "iter0labels.npy"), allow_pickle=True)
    
    return labels, alignment

In [ ]:
# STAligner
# sec_lists = ['151507_151508', '151508_151509', '151509_151510', '151669_151670', '151670_151671', '151671_151672', '151673_151674', '151674_151675', '151675_151676']
sec_lists = ['151507_151508', '151508_151509', '151509_151510', '151669_151670', '151670_151671', '151671_151672', '151673_151674', '151674_151675', '151675_151676']
# sec_lists = ['151507_151508']
for sec in sec_lists:

    # paste2
    labels, alignment = load_paste(path_='../samples/paste', sec=sec)
    res1 = cal_layer_based_alignment_result(alignment, labels)

In [ ]:
import matplotlib.pyplot as plt

#### feel free to adjust this to plot the figures for better comparison
cmap = ['#808000', '#000075', '#ffd8b1', '#9A6324', '#a9a9a9', '#4363d8', "#808080", "#fffac8", "#469990"]
order = ["STAligner", "PASTE2", "PASTE", "PRECAST", "SPACEL", "DeepST", "SPIRAL", "STalign", "GPSA"]

def plot(res0, res1, res2, res3, res4, res5, res6, res7, res8, sec, cmap):    
    index = [0,1,2,3,4,5,6]
    df = pd.DataFrame(data={'STAligner': res0, 'PASTE2': res1, 'PASTE': res2, 'PRECAST': res3, 'SPACEL': res4, 'DeepST':res5, 'SPIRAL':res6, 'STalign':res7, 'GPSA':res8}, index=index)
    my_colors = cmap

    # Reorder columns based on the accuracy of the first row
    sorted_columns = df.iloc[0].sort_values(ascending=False).index

    # Sort the colormap to match the sorted columns
    sorted_columns_with_colors = sorted(zip(df.columns, cmap), key=lambda x: df.loc[index[0], x[0]], reverse=True)
    sorted_columns, my_colors = zip(*sorted_columns_with_colors)

    # Reorder the DataFrame columns
    df = df[list(sorted_columns)]

    
    # df.plot.hist(column=["age"], by="gender", figsize=(10, 8))
    ax = df.plot.bar(rot=0, color=my_colors, width=0.9, xlim=(0,6), figsize=(30, 8), legend=True)
    # ax.set_xlim(-0.5, 2*len(df) - 0.5)
    # ax.set_xticks(range(0, 2*len(df), 2), index)
    ax.legend(fontsize=30)
    plt.xlabel("Layer shift")
    plt.ylabel("Layer-wise alignment accuracy", fontsize=35)
    
    yticks = np.arange(0, 1.0, 0.1)
    plt.yticks(yticks, fontsize=45)

    plt.title('DLPFC ' + sec.split('_')[0]+"-"+sec.split('_')[1])

    plt.savefig(os.path.join('./', "LAA" + sec + "barplot_ordered.pdf"), bbox_inches="tight")